# COPA Demo

In [7]:
import os

TASK_NAME = "COPA"
AUX_TASK_NAME = "SWAG"
BERT_MODEL = "bert-large-cased"

dataloader_config = {
    "batch_size": 4,
    "data_dir": os.environ["SUPERGLUEDATA"],
    "splits": ["train", "valid"],
}

trainer_config = {
    "lr": 1e-5,
    "optimizer": "adam",
    "n_epochs": 10,
    "conter_unit": "epochs",
    "evaluation_freq": 0.25,
}

### Apply auxiliary task

In [ ]:
from snorkel.mtl.model import MultitaskModel

from dataloaders import get_dataloaders
from superglue_tasks import task_funcs


dataloaders = get_dataloaders(
    task_name=AUX_TASK_NAME,
    tokenizer_name=BERT_MODEL,
    **dataloader_config
)

task = task_funcs[AUX_TASK_NAME](BERT_MODEL)
model = MultitaskModel(tasks=[task])

In [ ]:
# Start training on SWAG
from snorkel.mtl.trainer import Trainer

trainer = Trainer(**trainer_config)
trainer.train_model(model, dataloaders)
# Kill it

### Load primary task data

In [10]:
from dataloaders import get_dataloaders

dataloaders = get_dataloaders(
    task_name=TASK_NAME,
    tokenizer_name=BERT_MODEL,
    **dataloader_config
)

In [3]:
from superglue_tasks import task_funcs

task = task_funcs[TASK_NAME](BERT_MODEL)

100%|██████████| 1242874899/1242874899 [01:57<00:00, 10594637.26B/s]


### Fine-tune double pre-trained model (BERT + SWAG)

In [6]:
from snorkel.mtl.model import MultitaskModel

# Load SWAG model weights

In [ ]:
from snorkel.mtl.trainer import Trainer

trainer = Trainer(**trainer_config)
trainer.train_model(model, dataloaders)

In [ ]:
# Kill after you start training
# Load saved fine-tuned model

### Evaluate

In [ ]:
# Model score
# Pretty print results

### Compare to non-pretrained model 

In [ ]:
# Load non-slice model
# Score
# See the gap